In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 36.5 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
import math
import numpy as np
from scipy.linalg import block_diag
from scipy import optimize

In [ ]:
# defing parameters
wires =n_qubits = 5
depth = 10

In [ ]:
dev = qml.device("default.qubit", wires = wires)

In [ ]:
# Set parameters
delta = 1.0
Omega = 1.0
gamma = 1.0
Delta_t = 0.01

In [ ]:
# Exact vector a
alpha = np.sqrt(2)/2*delta*Delta_t
beta = (1-gamma*Delta_t/2)

In [ ]:
# Pauli operators
I = np.eye(2)
X = np.array([[0,1],[1,0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
Hadamard = np.array([[1, 1], [1, -1]]) / np.sqrt(2)

In [ ]:
# The non-Hermitian operator H

H0 = np.kron(I,1j*Hadamard)
H1 = np.kron(I,-Z)
H2 = np.kron((-1j * alpha * Hadamard + beta * I) / np.sqrt(alpha**2 + beta**2), I)
H3 = np.kron(-Z,I)
H4 = np.kron(X,X)
H5 = np.kron(X,-1j*Y)
H6 = np.kron(-1j*Y,X)
H7 = np.kron(-Y,Y)

In [ ]:
# The non-Hermitian operator Q

I_4 = np.eye(4)

Q0 = I_4 + Delta_t*H0
Q1 = I_4 + Delta_t*H1
Q2 = I_4 + Delta_t*H2
Q3 = I_4 + Delta_t*H3
Q4 = I_4 + Delta_t*H4
Q5 = I_4 + Delta_t*H5
Q6 = I_4 + Delta_t*H6
Q7 = I_4 + Delta_t*H7

In [ ]:
# Multi-controlled unitary
Lambda_Q = block_diag(Q0, Q1, Q2, Q3, Q4, Q5, Q6, Q7)
print(Lambda_Q)

[[ 1.  +0.00707107j  0.  +0.00707107j  0.  +0.j         ...
   0.  +0.j          0.  +0.j          0.  +0.j        ]
 [ 0.  +0.00707107j  1.  -0.00707107j  0.  +0.j         ...
   0.  +0.j          0.  +0.j          0.  +0.j        ]
 [ 0.  +0.j          0.  +0.j          1.  +0.00707107j ...
   0.  +0.j          0.  +0.j          0.  +0.j        ]
 ...
 [ 0.  +0.j          0.  +0.j          0.  +0.j         ...
   1.  +0.j         -0.01+0.j          0.  +0.j        ]
 [ 0.  +0.j          0.  +0.j          0.  +0.j         ...
  -0.01+0.j          1.  +0.j          0.  +0.j        ]
 [ 0.  +0.j          0.  +0.j          0.  +0.j         ...
   0.  +0.j          0.  +0.j          1.  +0.j        ]]


In [ ]:
@qml.qnode(dev,diff_method="backprop")
def circuit(weights):
  # since depth 4 was used
  Uq_theta = qml.matrix(qml.QNode(ansatz_layer, dev))
  # qml.state() applies Ua to Ini State, which gives Appro_a
  return Uq_theta

In [ ]:
def accuracy(labels, predictions):
  state0 = qml.math.dm_from_state_vector(labels)
  state1 = qml.math.dm_from_state_vector(predictions)
  return qml.math.fidelity(state0, state1)

In [ ]:
dev = qml.device("default.qubit",wires=wires)
dev

<default.qubit device (wires=5) at 0x7ad94c2e68c0>

In [ ]:
def variational_classifier(weights):
    # weights are thetas
    # np.real(Ua|0>)
    return circuit(weights)

In [ ]:
def g_beta_loss(Uq_theta):

  UqFunction = np.abs(1-1/(32**2) * np.trace(np.matmul(np.transpose(Uq_theta),Lambda_Q))**2)
  return UqFunction


In [ ]:
def cost(weights):
    # X is our preparation state
    Uq_theta = variational_classifier(weights)
    return g_beta_loss(Uq_theta)

In [ ]:
#UqLay
def ansatz_layer(layer_weights,depth=depth, n_qubits=n_qubits):
    cntr = 0
    for dep in range(depth):
        for wire in range(n_qubits):
          qml.RZ(layer_weights[wire+cntr], wires = wire)
          qml.RY(layer_weights[wire+1+cntr], wires = wire)
          qml.RZ(layer_weights[wire+2+cntr], wires = wire)
          cntr += 3

        qml.CRY(layer_weights[cntr], wires = [0,1])
        qml.CRY(layer_weights[cntr+1], wires = [2,3])
        qml.CRY(layer_weights[cntr+2], wires = [1,2])
        qml.CRY(layer_weights[cntr+3], wires = [4,0])
        cntr+4


In [ ]:
#no of params
param_num = depth*(3*n_qubits + 4)
weights = 4*np.pi*np.random.rand(param_num)
mat= qml.matrix(ansatz_layer, wire_order=wires)
print(mat)

<function TransformDispatcher._qfunc_transform.<locals>.qfunc_transformed at 0x7ad94c04e200>


In [ ]:
result= optimize.minimize(cost,weights, options={'maxiter':2048})

QuantumFunctionError: A quantum function must return either a single measurement, or a nonempty sequence of measurements.